In [120]:
import pandas
import FinanceDataReader as fdr
import talib

df = fdr.DataReader("BTC/KRW", "2018", "2022")
df = df.sort_index(ascending=False).astype('float')
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-31,56159392.0,57640380.0,54526892.0,55052808.0,55052808.0,4.395785e+13
2021-12-30,55076504.0,57034616.0,54534512.0,56169816.0,56169816.0,3.177268e+13
2021-12-29,56612400.0,57101840.0,54733608.0,55022132.0,55022132.0,3.559873e+13
2021-12-28,60139212.0,60139212.0,56335904.0,56570776.0,56570776.0,3.974003e+13
2021-12-27,60226492.0,61667020.0,59894500.0,60092408.0,60092408.0,2.886446e+13


In [139]:
df2 = fdr.DataReader("ETH/KRW", "2018", "2022")
df2 = df2.sort_index(ascending=False).astype('float')
df2.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-31,4421173.5,4526428.5,4327966.0,4378208.5,4378208.5,1.683131e+13
2021-12-30,4303019.0,4486523.5,4256652.5,4421676.0,4421676.0,1.538883e+13
2021-12-29,4514164.5,4541509.0,4279977.5,4298650.0,4298650.0,1.862620e+13
2021-12-28,4791141.0,4791141.0,4480185.0,4518273.5,4518273.5,2.056458e+13
2021-12-27,4818757.0,4897048.0,4787255.5,4791152.5,4791152.5,1.355671e+13


In [106]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import talib
from backtesting.test import SMA, GOOG


def optim_func(series):

    if series["# Trades"] < 10:
        return -1

    return series["Equity Final [$]"] / series["Exposure Time [%]"]

#전략 설정
class RSI(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14


    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
    
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()

        elif crossover(self.lower_bound, self.rsi):
            self.buy()

In [107]:
bt = Backtest(df, RSI, cash=50000, commission=0.0015)
stats = bt.run()
stats

C:\Users\kco79\AppData\Local\Temp\ipykernel_11720\2680903563.py:1: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, RSI, cash=50000, commission=0.0015)


Start                     2013-01-02 00:00:00
End                       2021-12-31 00:00:00
Duration                   3285 days 00:00:00
Exposure Time [%]                   27.040141
Equity Final [$]                115287.644221
Equity Peak [$]                 115287.644221
Return [%]                         130.575288
Buy & Hold Return [%]              703.162432
Return (Ann.) [%]                    9.731246
Volatility (Ann.) [%]               17.647827
Sharpe Ratio                         0.551413
Sortino Ratio                        0.910914
Calmar Ratio                         0.406672
Max. Drawdown [%]                  -23.929003
Avg. Drawdown [%]                   -3.458704
Max. Drawdown Duration      661 days 00:00:00
Avg. Drawdown Duration       53 days 00:00:00
# Trades                                    6
Win Rate [%]                            100.0
Best Trade [%]                      29.252693
Worst Trade [%]                      5.598527
Avg. Trade [%]                    

In [103]:
bt.plot()

Row(id='18138', ...)

In [34]:
stats = bt.optimize(
                    upper_bound = range(55, 85, 5),
                    lower_bound = range(10, 45, 5),
                    rsi_window = 14,
                    maximize="Return [%]",
                    constraint = lambda param: param.upper_bound > param.lower_bound,
                    )
                    
print(stats)

  0%|          | 0/14 [00:00<?, ?it/s]

Start                     2010-01-04 00:00:00
End                       2021-12-30 00:00:00
Duration                   4378 days 00:00:00
Exposure Time [%]                   78.291695
Equity Final [$]                    369392.35
Equity Peak [$]                     375792.35
Return [%]                           638.7847
Buy & Hold Return [%]              383.930779
Return (Ann.) [%]                   18.547244
Volatility (Ann.) [%]                27.77446
Sharpe Ratio                         0.667781
Sortino Ratio                        1.202307
Calmar Ratio                         0.542919
Max. Drawdown [%]                  -34.162107
Avg. Drawdown [%]                   -4.224593
Max. Drawdown Duration     1164 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                     274.438342
Worst Trade [%]                     14.675149
Avg. Trade [%]                    

In [38]:

bt.plot()
stats._strategy

<Strategy RSI(upper_bound=80,lower_bound=25,rsi_window=14)>

In [35]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,3,403,1445,14441.63,27700.0,39775.11,0.918066,2011-08-12,2015-11-02,1543 days
1,4,1497,2715,21792.64,81600.0,239229.44,2.744383,2016-01-18,2021-01-05,1814 days
2,4,2905,2961,68803.05,78900.0,40387.80,0.146751,2021-10-13,2021-12-30,78 days


In [98]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib

def optim_func(series):

    if series["# Trades"] < 10:
        return -1

    return series["Equity Final [$]"] / series["Exposure Time [%]"]

#전략 설정
class RSI2(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14


    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        self.rma = self.I(talib.SMA, self.rsi, self.rsi_window)
        
    def next(self):
        if (crossover(self.rsi, self.upper_bound) and self.rma > 55):
                self.position.close()
                self.sell()

        elif (crossover(self.lower_bound, self.rsi) and self.rma < 45):
                self.position.close()
                self.buy()

In [99]:
bt = Backtest(df, RSI2, cash=50000000)
stats = bt.run()

print(stats)

Start                     2013-01-02 00:00:00
End                       2021-12-31 00:00:00
Duration                   3285 days 00:00:00
Exposure Time [%]                   98.147331
Equity Final [$]              11015641.041506
Equity Peak [$]               56340752.016412
Return [%]                         -77.968718
Buy & Hold Return [%]              805.589954
Return (Ann.) [%]                  -15.477522
Volatility (Ann.) [%]               22.847139
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -80.690858
Avg. Drawdown [%]                  -19.299492
Max. Drawdown Duration     3066 days 00:00:00
Avg. Drawdown Duration      536 days 00:00:00
# Trades                                   70
Win Rate [%]                        25.714286
Best Trade [%]                      37.334848
Worst Trade [%]                    -22.199442
Avg. Trade [%]                    

C:\Users\kco79\AppData\Local\Temp\ipykernel_11720\2490821474.py:1: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, RSI2, cash=50000000)


In [100]:
bt.plot()

Row(id='16817', ...)

In [ ]:
stats._trades

In [244]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import talib

class SmaCross(Strategy):
    n1 = 10
    n2 = 20
    
    def init(self):
        self.sma1 = self.I(talib.SMA, self.data.Close, self.n1)
        self.sma2 = self.I(talib.SMA, self.data.Close, self.n2)
    
    def next(self):
        if crossover(self.sma1, self.sma2):
            # self.position.close()
            self.buy()

        # elif crossover(self.sma2, self.sma1):
        elif crossover(self.sma2, self.sma1) or self.position.pl_pct < -0.1:
            self.position.close()
            # self.sell()
        
        # elif self.position.pl_pct > -

In [245]:
bt = Backtest(df, SmaCross, cash=50000000, commission=.0015)
stats = bt.run()
stats

C:\Users\kco79\AppData\Local\Temp\ipykernel_11720\264790415.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, SmaCross, cash=50000000, commission=.0015)
C:\Users\kco79\AppData\Local\Temp\ipykernel_11720\264790415.py:1: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, SmaCross, cash=50000000, commission=.0015)


Start                     2018-01-01 00:00:00
End                       2021-12-31 00:00:00
Duration                   1460 days 00:00:00
Exposure Time [%]                   54.893908
Equity Final [$]              393152435.33975
Equity Peak [$]               578103706.55775
Return [%]                         686.304871
Buy & Hold Return [%]              278.413169
Return (Ann.) [%]                   67.395775
Volatility (Ann.) [%]               84.276239
Sharpe Ratio                         0.799701
Sortino Ratio                        2.263336
Calmar Ratio                         1.612662
Max. Drawdown [%]                  -41.791622
Avg. Drawdown [%]                   -8.262761
Max. Drawdown Duration      313 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   36
Win Rate [%]                        44.444444
Best Trade [%]                     112.539752
Worst Trade [%]                    -12.959952
Avg. Trade [%]                    

In [232]:
bt.plot()

Row(id='32061', ...)

In [ ]:
stats._trades

In [237]:
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Max. Drawdown [%]',
                    constraint=lambda param: param.n1 < param.n2)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

Start                     2018-01-01 00:00:00
End                       2021-12-31 00:00:00
Duration                   1460 days 00:00:00
Exposure Time [%]                   49.075975
Equity Final [$]                482501235.246
Equity Peak [$]                 600648859.246
Return [%]                          865.00247
Buy & Hold Return [%]              278.413169
Return (Ann.) [%]                   76.182853
Volatility (Ann.) [%]                89.02965
Sharpe Ratio                         0.855702
Sortino Ratio                        2.533102
Calmar Ratio                         2.186708
Max. Drawdown [%]                  -34.839054
Avg. Drawdown [%]                   -8.807025
Max. Drawdown Duration      401 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   17
Win Rate [%]                        47.058824
Best Trade [%]                     341.248518
Worst Trade [%]                     -15.79777
Avg. Trade [%]                    

In [238]:
stats._strategy

<Strategy SmaCross(n1=5,n2=65)>

In [239]:
bt.plot()

Row(id='33123', ...)

In [173]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,4,52,72,1.147541e+07,9826042.0,-6.597481e+06,-0.143731,2018-02-22,2018-03-14,20 days
1,4,109,136,8.849305e+06,9010275.0,6.438797e+05,0.018190,2018-04-20,2018-05-17,27 days
2,5,190,221,7.508280e+06,7372147.5,-6.806603e+05,-0.018131,2018-07-10,2018-08-10,31 days
3,5,244,256,8.031719e+06,7301429.0,-3.651448e+06,-0.090926,2018-09-02,2018-09-14,12 days
4,5,279,290,7.472501e+06,7381187.5,-4.565673e+05,-0.012220,2018-10-07,2018-10-18,11 days
5,5,297,298,7.395277e+06,7349312.0,-2.298240e+05,-0.006215,2018-10-25,2018-10-26,1 days
6,5,299,303,7.394696e+06,7216564.0,-8.906622e+05,-0.024089,2018-10-27,2018-10-31,4 days
7,8,359,381,4.297093e+06,4087925.5,-1.673344e+06,-0.048677,2018-12-26,2019-01-17,22 days
8,8,410,528,4.091222e+06,9640497.0,4.439420e+07,1.356386,2019-02-15,2019-06-13,118 days
9,7,531,566,1.050619e+07,12675453.0,1.518482e+07,0.206474,2019-06-16,2019-07-21,35 days


In [141]:
bt = Backtest(df2, SmaCross, cash=50000000, commission=.0015)
stats = bt.run()
stats

C:\Users\kco79\AppData\Local\Temp\ipykernel_11720\133695247.py:1: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df2, SmaCross, cash=50000000, commission=.0015)


Start                     2018-01-01 00:00:00
End                       2021-12-31 00:00:00
Duration                   1460 days 00:00:00
Exposure Time [%]                   54.688569
Equity Final [$]             726895333.988875
Equity Peak [$]             1024476753.462375
Return [%]                        1353.790668
Buy & Hold Return [%]              431.945517
Return (Ann.) [%]                    95.17606
Volatility (Ann.) [%]              150.670128
Sharpe Ratio                         0.631685
Sortino Ratio                        2.123084
Calmar Ratio                         1.802914
Max. Drawdown [%]                  -52.790118
Avg. Drawdown [%]                  -12.730338
Max. Drawdown Duration      375 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   37
Win Rate [%]                        48.648649
Best Trade [%]                     125.797598
Worst Trade [%]                    -26.451326
Avg. Trade [%]                    

In [142]:
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

Start                     2018-01-01 00:00:00
End                       2021-12-31 00:00:00
Duration                   1460 days 00:00:00
Exposure Time [%]                   52.429843
Equity Final [$]            1247561114.325422
Equity Peak [$]             1451806842.325422
Return [%]                        2395.122229
Buy & Hold Return [%]              431.945517
Return (Ann.) [%]                  123.374654
Volatility (Ann.) [%]              167.410741
Sharpe Ratio                         0.736958
Sortino Ratio                         2.82555
Calmar Ratio                         2.617332
Max. Drawdown [%]                   -47.13756
Avg. Drawdown [%]                  -12.274754
Max. Drawdown Duration      371 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                   17
Win Rate [%]                        64.705882
Best Trade [%]                     258.849745
Worst Trade [%]                    -20.636471
Avg. Trade [%]                    

In [143]:
bt.plot()

Row(id='23565', ...)

In [144]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,76,111,145,6.495808e+05,6.326578e+05,-1.286151e+06,-0.026052,2018-04-22,2018-05-26,34 days
1,202,314,315,2.405933e+05,2.394137e+05,-2.382949e+05,-0.004903,2018-11-11,2018-11-12,1 days
2,369,361,388,1.311830e+05,1.322983e+05,4.115462e+05,0.008502,2018-12-28,2019-01-24,27 days
3,325,413,560,1.500608e+05,2.684091e+05,3.846321e+07,0.788669,2019-02-18,2019-07-15,147 days
4,346,626,634,2.518776e+05,1.998990e+05,-1.798462e+07,-0.206365,2019-09-19,2019-09-27,8 days
5,320,649,652,2.163494e+05,2.213621e+05,1.604073e+06,0.023170,2019-10-12,2019-10-15,3 days
6,318,675,687,2.226853e+05,2.105904e+05,-3.846184e+06,-0.054314,2019-11-07,2019-11-19,12 days
7,403,740,800,1.664504e+05,2.379101e+05,2.879825e+07,0.429315,2020-01-11,2020-03-11,60 days
8,458,838,912,2.090453e+05,2.712122e+05,2.847246e+07,0.297385,2020-04-18,2020-07-01,74 days
9,433,919,930,2.866321e+05,2.834047e+05,-1.397450e+06,-0.011260,2020-07-08,2020-07-19,11 days
